In [11]:
from peft import PeftConfig, PeftModel
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import pipeline, logging

In [2]:
peft_model_id = "serkanarslan/mistral-7b-mini-ft"

In [3]:
config = PeftConfig.from_pretrained(peft_model_id)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict = True,
    load_in_4bit = True,
    #load_in_8bit_fp32_cpu_offload=True,
    device_map = "auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
    padding_size = "right",
    add_eos_token = True,

)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
fine_tune_model = PeftModel.from_pretrained(
    model, peft_model_id
)

c:\Users\SERKAN\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SERKAN\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(
    task = "text-generation",
    model = fine_tune_model,
    tokenizer = tokenizer,
    eos_token_id = model.config.eos_token_id,
    max_new_tokens = 100
)

In [13]:
prompt = """
What is a Python? Here is some context: Python ise a high-level programming language.
"""

In [15]:
result = pipe(f"<s>[INST]{prompt}[/INST]")

c:\Users\SERKAN\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\nn\modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


In [20]:
generated = result[0]["generated_text"]
print(generated[generated.find("[/INST]")+8:])

Python is a high-level programming language. It is a general-purpose language, meaning it can be used to create a wide variety of programs, including web applications, desktop applications, and scientific and numerical programs. Python is known for its readability and simplicity, making it a popular choice for beginners and experienced programmers alike. It is also known for its large and active community, which contributes to the development and maintenance of the language. Python is often used in combination with other programming
